In [ ]:
import azureml.core
from azureml.core import Workspace, Datastore, Dataset

mlonazure_ws = Workspace.from_config()

mlonazure_ds = mlonazure_ws.get_default_datastore()

print('Workspace Name: ' + mlonazure_ws.name, 
      'Resource Group: ' + mlonazure_ws.resource_group,
      'Default Storage Account Name: ' + mlonazure_ds.account_name,
      'AzureML Core Version: ' + azureml.core.VERSION,
      sep = '\n')

In [ ]:
from azureml.pipeline.core import PipelineEndpoint
  
pipelineEndPoint = PipelineEndpoint.get(workspace=mlonazure_ws, name="HelloWorldEndPoint")
pipelineEndPoint

In [ ]:
pipelineEndPoint.endpoint

In [ ]:
from azureml.core import Experiment 

experiment_name = 'Pipeline-Helloworld'
experiment = Experiment(workspace = mlonazure_ws, name = experiment_name)

experiment.tag('Author', 'mlonazure')
experiment.tag('Department', 'Intro')
experiment.tag('Executor', 'Experiment -> EndPoint')

exp_pipelinerun = experiment.submit(pipelineEndPoint)

exp_pipelinerun.wait_for_completion()

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
import requests

auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

restEndPointForPipeline = pipelineEndPoint.endpoint

print("You can perform HTTP POST on URL {} to trigger this pipeline".format(restEndPointForPipeline))

# specify the param when running the pipeline
response = requests.post(restEndPointForPipeline, 
                         headers=aad_token, 
                         json={"ExperimentName": "Pipeline-Helloworld",
                               "RunSource": "SDK",
                               "ParameterAssignments": {"pipeline_arg": 45}})

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception('Received bad response from the endpoint: {}\n'
                    'Response Code: {}\n'
                    'Headers: {}\n'
                    'Content: {}'.format(restEndPointForPipeline, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)